# 🚀 Developed by Dr. Mohamed Amine Ferrag @2025 - Script for benchmarking LLM using AgentDrive-MCQ

In [ ]:

# ============================
# 🚀 Developed by Dr. Mohamed Amine Ferrag @2025
# ============================

# Requirements: openai, pandas, matplotlib, tqdm

#!pip -qU install openai pandas matplotlib tqdm

import os, json, re, time, random, queue, threading
from pathlib import Path
from datetime import datetime, timezone
from typing import Dict, Any, List, Optional
import pandas as pd
import numpy as np
from google.colab import drive
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt

# ---------- User Config ----------
DRIVE_JSON_DIR = "/content/drive/MyDrive/DriveBench/"          # Folder containing MCQ JSONs

OPENROUTER_MODEL = os.getenv("OPENROUTER_MODEL", " ").strip()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY", " ").strip()

visible_model_name = OPENROUTER_MODEL.replace("/", "／")  # full-width slash U+FF0F

RESULTS_CSV = f"/content/drive/MyDrive/DriveBench/Evaluations/mcq_results_{visible_model_name}.csv"
STATS_CSV   = f"/content/drive/MyDrive/DriveBench/Evaluations/mcq_style_stats_{visible_model_name}.csv"

# Concurrency & behavior
NUM_WORKERS = 20    # 🔧 threads for parallel LLM calls (tune per your rate limits)
BATCH_LIMIT = None  # e.g., 200 to process a subset of JSONs
DRY_RUN     = False # True = random answers
VERBOSE     = True  # True = print per-sample steps

# Progress heartbeat (every N rows processed)
PROGRESS_EVERY = 20

# ---------- Checks ----------
if not OPENROUTER_API_KEY and not DRY_RUN:
    raise RuntimeError("Missing OPENROUTER_API_KEY. Set via Colab secrets or os.environ.")

# ---------- Mount Drive ----------
drive.mount('/content/drive', force_remount=True)

# ---------- Client & helpers ----------
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=OPENROUTER_API_KEY)
print("Model:", OPENROUTER_MODEL)

LETTER_RE = re.compile(r"\b([ABCD])\b")

def _utc_now_iso():
    return datetime.now(timezone.utc).isoformat(timespec="seconds").replace("+00:00", "Z")

def list_jsons(folder: str) -> List[Path]:
    p = Path(folder)
    return sorted([x for x in p.rglob("*.json") if x.is_file()])

def read_json(path: Path) -> Dict[str, Any]:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def strict_prompt(desc: str, question: str, choices: List[str]) -> List[Dict[str, str]]:
    labels = ["A","B","C","D"]
    normalized = []
    for i, ch in enumerate(choices[:4]):
        s = str(ch).strip()
        if re.match(r"^[A-D][\s\-:]", s, flags=re.IGNORECASE):
            normalized.append(s)
        else:
            normalized.append(f"{labels[i]}- {s}")
    choice_block = "\n".join(normalized)

    system = (
        "You are a careful evaluator for multiple-choice driving safety questions. "
        "Return EXACTLY ONE CAPITAL LETTER from {A,B,C,D}. No words, no punctuation, no explanation."
    )
    user = (
        f"DESCRIPTION:\n{desc}\n\n"
        f"QUESTION:\n{question}\n\n"
        f"CHOICES (pick ONE):\n{choice_block}\n\n"
        "Answer format: A single capital letter from {A,B,C,D} only."
    )
    return [{"role":"system","content":system},{"role":"user","content":user}]


def call_openrouter_letter(messages: List[Dict[str, str]], model: str, max_retries=5, backoff_base=1.8) -> str:
    if DRY_RUN:
        return random.choice(list("ABCD"))
    for attempt in range(1, max_retries+1):
        try:
            resp = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0.0,
                max_tokens=16,
                top_p=1.0,
            )
            txt = (resp.choices[0].message.content or "").strip()
            m = LETTER_RE.search(txt)
            if m:
                return m.group(1).upper()
            if txt and txt[0].upper() in "ABCD":
                return txt[0].upper()
            return "?"
        except Exception as e:
            delay = min(30, (backoff_base ** attempt)) + random.uniform(0, 0.75)
            if VERBOSE:
                print(f"[retry] {e} — sleeping {delay:.1f}s (attempt {attempt})")
            time.sleep(delay)
    return "?"

def normalize_correct_choice(v: Any) -> str:
    if not v: return ""
    s = str(v).strip()
    m = LETTER_RE.search(s)
    return m.group(1).upper() if m else ""

def ensure_csv(path: str):
    p = Path(path)
    if not p.exists():
        df = pd.DataFrame(columns=[
            "timestamp","file","scenario_name","model","style",
            "answer","correct_choice","is_correct",
            "question","choice_A","choice_B","choice_C","choice_D"
        ])
        df.to_csv(p, index=False)

def already_done_set(path: str) -> set:
    p = Path(path)
    if not p.exists():
        return set()
    try:
        df = pd.read_csv(p, usecols=["file"])
        return set(df["file"].astype(str).tolist())
    except Exception:
        return set()

def clean_choice_text(s: str) -> str:
    return re.sub(r"^[A-D][\s\-:]\s*", "", str(s).strip(), flags=re.IGNORECASE)

# ---------- Load file list ----------
ensure_csv(RESULTS_CSV)
done = already_done_set(RESULTS_CSV)
json_files = list_jsons(DRIVE_JSON_DIR)
if BATCH_LIMIT is not None:
    json_files = json_files[:BATCH_LIMIT]

print(f"Found {len(json_files)} JSON files in {DRIVE_JSON_DIR}")
print(f"Already in CSV: {len(done)}")

# ---------- Writer thread (single writer, thread-safe) ----------
write_q: "queue.Queue[Dict[str, Any]]" = queue.Queue(maxsize=1000)
stop_signal = object()
csv_lock = threading.Lock()  # extra safety if needed

def writer_loop():
    # append rows in small batches
    buffer = []
    last_flush = time.time()
    FLUSH_EVERY_SEC = 3
    while True:
        item = write_q.get()
        if item is stop_signal:
            break
        buffer.append(item)
        # periodic flush or if buffer large
        if len(buffer) >= 64 or (time.time() - last_flush) >= FLUSH_EVERY_SEC:
            dfw = pd.DataFrame(buffer)
            with csv_lock:
                dfw.to_csv(RESULTS_CSV, mode="a", header=not Path(RESULTS_CSV).exists() or os.path.getsize(RESULTS_CSV)==0, index=False)
            buffer.clear()
            last_flush = time.time()
    # final flush
    if buffer:
        dfw = pd.DataFrame(buffer)
        with csv_lock:
            dfw.to_csv(RESULTS_CSV, mode="a", header=not Path(RESULTS_CSV).exists() or os.path.getsize(RESULTS_CSV)==0, index=False)

writer_thread = threading.Thread(target=writer_loop, daemon=True)
writer_thread.start()

# ---------- Worker function ----------
def process_file(path: Path) -> Dict[str, Any]:
    rel = str(path)
    # Step 1: read JSON
    try:
        obj = read_json(path)
        if VERBOSE: print(f"[read] {rel}")
    except Exception as e:
        return {"error": f"read_json failed: {e}", "file": rel}

    # Step 2: extract fields
    desc = obj.get("description", "").strip()
    question = obj.get("question", "").strip()
    choices = obj.get("choices", [])
    style = (obj.get("style") or "unknown").strip().lower()
    scenario_name = obj.get("scenario_name", path.stem)
    if not (desc and question and isinstance(choices, list) and len(choices) >= 4):
        return {"error": "missing fields", "file": rel}
    if VERBOSE:
        print(f"[prompt] {rel} | style={style} | scenario={scenario_name}")

    # Step 3: build prompt
    messages = strict_prompt(desc, question, choices)

    # Step 4: call LLM
    answer = call_openrouter_letter(messages, OPENROUTER_MODEL)
    if VERBOSE:
        print(f"[llm] {rel} → {answer}")

    # Step 5: correctness
    correct = normalize_correct_choice(obj.get("correct_choice"))
    is_correct = (answer == correct) if correct else ""

    # Step 6: assemble row
    row = {
        "timestamp": _utc_now_iso(),
        "file": rel,
        "scenario_name": scenario_name,
        "model": OPENROUTER_MODEL,
        "style": style,
        "answer": answer,
        "correct_choice": correct,
        "is_correct": is_correct,
        "question": question,
        "choice_A": clean_choice_text(choices[0]),
        "choice_B": clean_choice_text(choices[1]),
        "choice_C": clean_choice_text(choices[2]),
        "choice_D": clean_choice_text(choices[3]),
    }
    if VERBOSE:
        print(f"[write] {rel} | is_correct={is_correct}")
    return row

# ---------- Parallel execution ----------
to_process = [p for p in json_files if str(p) not in done]
print(f"Queued {len(to_process)} files for processing with {NUM_WORKERS} threads.")

processed = 0
errors = 0

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as ex:
    futures = {ex.submit(process_file, p): p for p in to_process}
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
        res = fut.result()
        if "error" in res:
            errors += 1
            if VERBOSE:
                print(f"[error] {res.get('file','?')}: {res['error']}")
        else:
            write_q.put(res)
            processed += 1
            if processed % PROGRESS_EVERY == 0:
                print(f"[progress] processed={processed}/{len(to_process)} | errors={errors}")

# Stop writer and join
write_q.put(stop_signal)
writer_thread.join()

print(f"✅ Done. Processed={processed}, Errors={errors}")
print(f"Results CSV → {RESULTS_CSV}")

# ---------- Accuracy by style ----------
if not Path(RESULTS_CSV).exists():
    raise FileNotFoundError(f"Results CSV not found: {RESULTS_CSV}")

df = pd.read_csv(RESULTS_CSV)

# Optional test subset for stats (NUM_SAMPLES)
NUM_SAMPLES = None  # set above if desired; leaving here for quick edits
if NUM_SAMPLES is not None:
    df = df.head(NUM_SAMPLES)
    print(f"⚙️ Stats computed on first {NUM_SAMPLES} rows.")

# Ensure columns
for col in ["style","is_correct"]:
    if col not in df.columns:
        raise ValueError(f"Missing column '{col}' in {RESULTS_CSV}")

# Normalize style
BUCKETS = {"physics","policy","hybrid","scenario","comparative"}
df["style"] = df["style"].astype(str).str.strip().str.lower()
df["style"] = df["style"].apply(lambda s: s if s in BUCKETS else "unknown")

# Coerce is_correct
def to_bool(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    if s in ("true","1","yes","y"):  return True
    if s in ("false","0","no","n"):  return False
    if x in (True, False):           return bool(x)
    return np.nan

df["is_correct_bool"] = df["is_correct"].apply(to_bool)

agg = (
    df.groupby("style", dropna=False)
      .agg(
          total   = ("is_correct_bool", "size"),
          answered= ("is_correct_bool", lambda s: int(s.notna().sum())),
          correct = ("is_correct_bool", lambda s: int((s == True).sum())),
          incorrect=("is_correct_bool", lambda s: int((s == False).sum())),
      )
      .reset_index()
)

agg["accuracy"] = (agg["correct"] / agg["answered"]).replace([np.inf, -np.inf], np.nan).round(4)

overall = pd.DataFrame([{
    "style": "ALL",
    "total": int(df.shape[0]),
    "answered": int(df["is_correct_bool"].notna().sum()),
    "correct": int((df["is_correct_bool"] == True).sum()),
    "incorrect": int((df["is_correct_bool"] == False).sum()),
}])
overall["accuracy"] = (overall["correct"] / overall["answered"]).round(4)

summary = pd.concat([agg, overall], ignore_index=True)
summary.to_csv(STATS_CSV, index=False)

print(summary)
print("\n📁 Saved summary to:", STATS_CSV)

# Optional chart
view = summary[summary["style"].isin(["physics","policy","hybrid","scenario","comparative"])]
if not view.empty:
    plt.figure()
    plt.bar(view["style"], view["accuracy"])
    plt.title("Accuracy by Scenario Style")
    plt.xlabel("Style")
    plt.ylabel("Accuracy")
    plt.show()


Mounted at /content/drive
Model: arcee-ai/afm-4.5b
Found 200 JSON files in /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced
Already in CSV: 0
Queued 200 files for processing with 20 threads.


Processing:   0%|          | 0/200 [00:00<?, ?it/s]

[read] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveCarHairpinConstruction_v1_MCQ_comparative_checked.json
[prompt] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveCarHairpinConstruction_v1_MCQ_comparative_checked.json | style=comparative | scenario=AggressiveCarHairpinConstruction_v1
[read] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveRainyServiceRoadEntryExit_v1_MCQ_physics_checked.json
[prompt] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveRainyServiceRoadEntryExit_v1_MCQ_physics_checked.json | style=physics | scenario=AggressiveRainyServiceRoadEntryExit_v1
[read] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveWeaveBridgeIce_v1_MCQ_physics_checked.json
[prompt] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveWeaveBridgeIce_v1_MCQ_physics_checked.json | style=physics | scenario=AggressiveWeaveBridgeIce_v1
[read] /content/drive/MyDrive/Driv

Processing:   4%|▎         | 7/200 [00:00<00:19, 10.08it/s]

[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveCarHairpinConstruction_v1_MCQ_hybrid_checked.json → D
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveCarHairpinConstruction_v1_MCQ_hybrid_checked.json | is_correct=True
[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveWeaveFloodedRamp_v1_MCQ_scenario.json → C
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveWeaveFloodedRamp_v1_MCQ_scenario.json | is_correct=True
[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveWeaveBridgeIcyEasy_MCQ_scenarioChecked.json → C
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveWeaveBridgeIcyEasy_MCQ_scenarioChecked.json | is_correct=True
[read] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/AggressiveWeaveHailstormCloverLeafYield_v1_MCQ_comparative.json
[prompt] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/

...

[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/BrakeFailureDownhillNoviceLearner_v1_MCQ_comparative.json → C
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/BrakeFailureDownhillNoviceLearner_v1_MCQ_comparative.json | is_correct=True
[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/BrakeFailureDownhillPoliteSnowstormLC_v1_MCQ_policy.json → D
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/BrakeFailureDownhillPoliteSnowstormLC_v1_MCQ_policy.json | is_correct=True
[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/BrakeFailureDownhillRedLightMerge_v1_MCQ_physics.json → B
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/BrakeFailureDownhillRedLightMerge_v1_MCQ_physics.json | is_correct=True
[read] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/ChildBallChase_NightHighway_v1_MCQ_scenario.json
[prompt] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/C

Processing:  96%|█████████▌| 191/200 [00:04<00:00, 62.16it/s]

[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/ChildChaseBall_SportsCarThunderstormRoundabout_v1_MCQ_comparative.json → D
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/ChildChaseBall_SportsCarThunderstormRoundabout_v1_MCQ_comparative.json | is_correct=False
[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/BrakeFailureDownhillThrillSeekerGravelNarrowUrbanStreet_v1_MCQ_physics.json → B
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/BrakeFailureDownhillThrillSeekerGravelNarrowUrbanStreet_v1_MCQ_physics.json | is_correct=True
[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/BrakeFailureDownhill_Inexperienced_DawnDriving_UphillSlope_SafeReactEmergencyVehicle_v1_MCQ_physics.json → C
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/BrakeFailureDownhill_Inexperienced_DawnDriving_UphillSlope_SafeReactEmergencyVehicle_v1_MCQ_physics.json | is_correct=True
[llm] /content/drive/

Processing: 100%|██████████| 200/200 [00:04<00:00, 42.19it/s]

[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/Jaywalk_Tunnel_Merge_v1_MCQ_comparative.json → D
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/Jaywalk_Tunnel_Merge_v1_MCQ_comparative.json | is_correct=False
[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/DeliveryTruckDoublePark_NightGlare_v1_MCQ_scenario.json → B
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/DeliveryTruckDoublePark_NightGlare_v1_MCQ_scenario.json | is_correct=False
[llm] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/CurveCutIn_Elderly_v1_MCQ_policy.json → C
[write] /content/drive/MyDrive/DriveBench/mcq_jsons_MCQ_200_balanced/CurveCutIn_Elderly_v1_MCQ_policy.json | is_correct=False
[progress] processed=200/200 | errors=0
✅ Done. Processed=200, Errors=0
Results CSV → /content/drive/MyDrive/DriveBench/Evaluations/mcq_results_arcee-ai／afm-4.5b.csv
         style  total  answered  correct  incorrect  accuracy
0  comparative   